### Question-Answer using Langchain

In [1]:
# !pip3 install langchain

In [2]:
# !pip install openai

In [3]:
# !pip3 install openai chromadb
# !pip3 install tiktoken

In [4]:
# !pip3 install unstructured
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip3 install pdfminer.six

In [5]:
import os
import sys
import glob
import re
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
# from langchain.document_loaders import WebBaseLoader, BasePDFLoader

#### API Key

In [6]:
os.environ["OPENAI_API_KEY"] = ""

#### Parameters

In [7]:
pdfs_path = os.path.join(os.getcwd(), '../data/pdfs')

#### Test

###### 1.Web

In [8]:
loader = WebBaseLoader("https://www.forbes.com/sites/mattnovak/2023/07/10/google-confirms-bug-sending-less-traffic-to-online-news-publishers")
index = VectorstoreIndexCreator().from_loaders([loader])

In [10]:
index.query("Can you summarise this article in less than 200 words?")

' Google has confirmed that an issue with Google News has been identified that is causing a drop in traffic to news publishers on Monday. The issue is not yet resolved and Google has not provided any further information. This serves as a reminder of the power of Google, as when it sneezes, the internet gets a cold. The article discusses how the new version of Google Search essentially plagiarizes answers, giving users little reason to click on websites for more information.'

###### 2.PDF

In [8]:
all_pdfs = glob.glob(os.path.join(pdfs_path, '*.pdf'))

In [9]:
loader = UnstructuredPDFLoader(file_path=all_pdfs[-1], mode='single')
index = VectorstoreIndexCreator().from_loaders([loader])

In [10]:
index.query("Can you summarise this article in less than 200 words?")

" Google is a holding company that was formerly known as Google. It has a core internet advertising business and other 'moonshots' such as autonomous vehicles and Verily Life Sciences. In March 2022, Google spun off its quantum computing technology group as a separate company. Google stock is strong in arti\x00cial intelligence, digital advertising, Google Cloud Platform, YouTube, consumer hardware products, and e-commerce. It has huge cash holdings and has shrugged off \x00nes from the European Union. Google has rolled out the Performance Max advertising platform to automate buying across its ad inventory. It has also acquired \x00rms such as Looker and Mandiant to boost its cloud business. Google stock faces more dif\x00cult year-over-year growth comparisons in 2023 as the coronavirus emergency fades."